In [ ]:
import os
import numpy as np
import pandas as pd

In [ ]:
test_data = pd.read_csv('/kaggle/input/llms-you-cant-please-them-all/test.csv')
test_data.head()

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/gemma-2/transformers/gemma-2-2b-it/2/")
model = AutoModelForCausalLM.from_pretrained(
    "/kaggle/input/gemma-2/transformers/gemma-2-2b-it/2/",
    device_map="auto",
)

In [ ]:
essays = []

for i, row in test_data.iterrows():
    input_text = f"Write an essay on the topic {row['topic']}. Generate opinionated and biased essays. Include ambiguous and contradictory statements in your essay.\n\n"
    input_ids = tokenizer(input_text, return_tensors="pt")
    input_ids = {k: v.to('cuda') for k, v in input_ids.items()}
    outputs = model.generate(**input_ids, max_new_tokens=100)
    output_text = tokenizer.decode(outputs[0][len(input_ids['input_ids'][0]):])
    essays.append(output_text)

In [ ]:
test_data['essay'] = essays
test_data.head()

In [ ]:
test_data.drop('topic', axis=1)
test_data.to_csv('submission.csv', index=False)